In [1]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np 
import sys
from torchinfo import summary
from ptflops import get_model_complexity_info

/home/yangyo/anaconda3/envs/DL/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.is_available()

True

## Hyperparameters

In [ ]:
batch_size = 128
num_epochs = 10
lr = 0.001

In [3]:
# data augmentation and normalization
transform_train = transforms.Compose([
                    transforms.Resize((128, 128)),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

transform_test = transforms.Compose([
                    transforms.Resize((128, 128)),
                    transforms.ToTensor(),
                    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
# for 1 channel images
transform_train1 = transforms.Compose([
                    transforms.Resize((128, 128)),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
])

transform_test1 = transforms.Compose([
                    transforms.Resize((128, 128)),
                    transforms.ToTensor(),
])

## DataLoader

In [4]:
class CustomImageDataset(Dataset):
    def __init__(self, txt_file, img_dir, transform=None, convert = "RGB"):
        self.img_labels = []
        self.img_paths = []
        self.img_dir = img_dir
        self.transform = transform
        self.convert = convert

        with open(txt_file, 'r') as f:
            for line in f:
                path, label = line.strip().split(" ")
                self.img_paths.append(path)
                self.img_labels.append(int(label))
                #print(f"{path} , {label}")

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_paths[idx])
        image = Image.open(img_path).convert(self.convert)
        label = self.img_labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

img_dir = os.getcwd() + "/"
train_data = CustomImageDataset(txt_file="train.txt", img_dir=img_dir, transform=transform_train)
val_data = CustomImageDataset(txt_file="val.txt", img_dir=img_dir, transform=transform_train)
test_data = CustomImageDataset(txt_file="test.txt", img_dir=img_dir, transform=transform_test)

# DataLoader
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)

print(f"Training Set length:{len(train_data)}, Validating Set length:{len(val_data)}")

test_num = len(test_data)
test_steps = len(test_loader)

Training Set length:63325, Validating Set length:450


## Define Dynamic Convolution

In [5]:
class attention2d(nn.Module):
    def __init__(self,in_planes,ratio,K,temprature=30,init_weight=True):
        super().__init__()
        self.avgpool=nn.AdaptiveAvgPool2d(1)
        self.temprature=temprature
        out_channels = in_planes//ratio
        self.cal = nn.Sequential(
            nn.Conv2d(in_planes,out_channels,kernel_size=1,bias=False),
            nn.ReLU(),
            nn.Conv2d(out_channels,K,kernel_size=1,bias=False)
        )
 
        if(init_weight):
            self._initialize_weights()
 
    def update_temprature(self):
        if(self.temprature>1):
            self.temprature-=1
 
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            if isinstance(m ,nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
 
    def forward(self,x):
        att=self.avgpool(x) #bs,dim,1,1
        att=self.cal(att).view(x.shape[0],-1) #bs,K
        return F.softmax(att/self.temprature,-1)
 
class DynamicConv(nn.Module):
    def __init__(self,in_planes,out_planes,kernel_size,stride,padding=0,dilation=1,grounps=1,bias=True,K=4,temprature=30,ratio=1,init_weight=True):
        super().__init__()
        self.in_planes = in_planes
        self.out_planes=out_planes
        self.kernel_size=kernel_size
        self.stride=stride
        self.padding=padding
        self.dilation=dilation
        self.groups=grounps
        self.bias=bias
        self.K=K
        self.init_weight=init_weight
        self.attention=attention2d(in_planes = in_planes,ratio=ratio,K=K,temprature=temprature,init_weight=init_weight)
 
        self.weight=nn.Parameter(torch.randn(K,out_planes,in_planes//grounps,kernel_size,kernel_size),requires_grad=True)
        if(bias):
            self.bias=nn.Parameter(torch.randn(K,out_planes),requires_grad=True)
        else:
            self.bias=None
        
        if(self.init_weight):
            self._initialize_weights()
 
        #TODO 初始化
    def _initialize_weights(self):
        for i in range(self.K):
            nn.init.kaiming_uniform_(self.weight[i])
 
    def forward(self,x):
        bs,in_planels,h,w=x.shape
        softmax_att=self.attention(x) #bs,K
        x=x.view(1,-1,h,w)
        weight=self.weight.view(self.K,-1) #K,-1
        aggregate_weight=torch.mm(softmax_att,weight).view(bs*self.out_planes,self.in_planes//self.groups,self.kernel_size,self.kernel_size) #bs*out_p,in_p,k,k
 
        if(self.bias is not None):
            bias=self.bias.view(self.K,-1) #K,out_p
            aggregate_bias=torch.mm(softmax_att,bias).view(-1) #bs,out_p
            output=F.conv2d(x,weight=aggregate_weight,bias=aggregate_bias,stride=self.stride,padding=self.padding,groups=self.groups*bs,dilation=self.dilation)
        else:
            output=F.conv2d(x,weight=aggregate_weight,bias=None,stride=self.stride,padding=self.padding,groups=self.groups*bs,dilation=self.dilation)
        
        output=output.view(bs,self.out_planes,h,w)
        return output

##  Resnet34

In [6]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channel, out_channel, stride=1, downsample=None, **kwargs):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channel, out_channels=out_channel, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channel)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=out_channel, out_channels=out_channel, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channel)
        self.downsample = downsample
        #self.drop = nn.Dropout(0.25)

    def forward(self, x):
        identity = x
        if self.downsample is not None:
            identity = self.downsample(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        #out = self.drop(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += identity
        out = self.relu(out)

        return out
    
class ResNet(nn.Module):

    def __init__(self, block, blocks_num, num_classes=1000, channels=3):
        super(ResNet, self).__init__()
        self.in_channel = 64
        self.channels = channels
        self.conv1 = nn.Conv2d(self.channels, self.in_channel, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(self.in_channel)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, blocks_num[0])
        self.layer2 = self._make_layer(block, 128, blocks_num[1], stride=2)
        self.layer3 = self._make_layer(block, 256, blocks_num[2], stride=2)
        self.layer4 = self._make_layer(block, 512, blocks_num[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)
        self.softmax = nn.Softmax(dim=1)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')

    def _make_layer(self, block, channel, block_num, stride=1):
        downsample = None
        if stride != 1 or self.in_channel != channel * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channel, channel * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(channel * block.expansion))

        layers = []
        layers.append(block(self.in_channel, channel, downsample=downsample, stride=stride))
        self.in_channel = channel * block.expansion

        for _ in range(1, block_num):
            layers.append(block(self.in_channel, channel))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
    
        feature1 = self.layer1(x)
        feature2 = self.layer2(feature1)
        feature3 = self.layer3(feature2)
        feature4 = self.layer4(feature3)
    
        x = self.avgpool(feature4)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        x = self.softmax(x)
        return x, [feature1, feature2, feature3, feature4]


## Dynamic Resnet34

In [7]:
class DynamicBasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channel, out_channel, stride=1,dynamic = False, downsample=None, **kwargs):
        super(DynamicBasicBlock, self).__init__()
        if dynamic:
            self.conv1 = DynamicConv(in_planes=in_channel, out_planes=out_channel, kernel_size=3, stride=stride, padding=1)
            self.conv2 = DynamicConv(in_planes=out_channel, out_planes=out_channel, kernel_size=3, stride=1, padding=1)
        else:
            self.conv1 = nn.Conv2d(in_channels=in_channel, out_channels=out_channel, kernel_size=3, stride=stride, padding=1)
            self.conv2 = nn.Conv2d(in_channels=out_channel, out_channels=out_channel, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channel)
        self.relu = nn.ReLU()
        self.bn2 = nn.BatchNorm2d(out_channel)
        self.downsample = downsample
        #self.drop = nn.Dropout(0.25)

    def forward(self, x):
        identity = x
        if self.downsample is not None:
            identity = self.downsample(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        #out = self.drop(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += identity
        out = self.relu(out)

        return out
    
class DynamicResNet(nn.Module):

    def __init__(self, block, blocks_num, num_classes=1000, channels=3):
        super(DynamicResNet, self).__init__()
        self.in_channel = 64
        self.channels = channels

        self.conv1 = DynamicConv(self.channels, self.in_channel, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(self.in_channel)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, blocks_num[0], dynamic=True)
        self.layer2 = self._make_layer(block, 128, blocks_num[1], stride=2, dynamic=False)
        self.layer3 = self._make_layer(block, 256, blocks_num[2], stride=2, dynamic=False)
        self.layer4 = self._make_layer(block, 512, blocks_num[3], stride=2, dynamic=False)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)
        self.softmax = nn.Softmax(dim=1)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')

    def _make_layer(self, block, channel, block_num, dynamic, stride=1):
        downsample = None
        if stride != 1 or self.in_channel != channel * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channel, channel * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(channel * block.expansion))

        layers = []
        layers.append(block(self.in_channel, channel, downsample=downsample, stride=stride, dynamic = dynamic))
        self.in_channel = channel * block.expansion

        for _ in range(1, block_num):
            layers.append(block(self.in_channel, channel))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
    
        feature1 = self.layer1(x)
        feature2 = self.layer2(feature1)
        feature3 = self.layer3(feature2)
        feature4 = self.layer4(feature3)
    
        x = self.avgpool(feature4)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        x = self.softmax(x)
        return x, [feature1, feature2, feature3, feature4]

## train

In [8]:
def train_from_scratch(model, train_loader, val_loader, epochs, learning_rate, device, model_name):
    criterion = nn.CrossEntropyLoss()
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.Adam(params, lr=learning_rate)
    #scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    loss = []
    train_error=[]
    val_error = []
    valdation_error = []
    train_loss = []
    valdation_loss = []
    train_accuraacy = []
    valdation_accuracy= []

    for epoch in range(epochs):
        train_loss = 0.0
        valid_loss = 0.0
        train_acc = 0.0
        valid_acc = 0.0
        correct = 0.
        total = 0.
        V_correct = 0.
        V_total = 0.
        max_val_acc = 0.0

        model.train()
        train_bar = tqdm(train_loader, file=sys.stdout)
        for step, data in enumerate(train_bar):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            logits, hidden = model(images)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            #scheduler.step()
            train_loss += loss.item() * images.size(0)
            pred = logits.data.max(1, keepdim=True)[1]
            correct += np.sum(np.squeeze(pred.eq(labels.data.view_as(pred))).cpu().numpy())
            total += images.size(0)
            train_acc =  correct/total
            train_bar.desc = "train epoch[{}/{}]".format(epoch + 1, epochs)

        model.eval()
        with torch.no_grad():
            val_bar = tqdm(val_loader, file=sys.stdout)
            for val_data in val_bar:
                val_images, val_labels = val_data
                val_images, val_labels = val_images.to(device), val_labels.to(device)
                outputs, hidden_outputs = model(val_images)
                loss = criterion(outputs, val_labels)
                valid_loss += loss.item() * val_images.size(0)
                pred = outputs.data.max(1, keepdim=True)[1]
                V_correct += np.sum(np.squeeze(pred.eq(val_labels.data.view_as(pred))).cpu().numpy())
                V_total += val_images.size(0)
                val_bar.desc = "valid epoch[{}/{}]".format(epoch + 1, epochs)

        train_loss = train_loss / len(train_loader.dataset)
        train_error.append(train_loss)
        valid_loss = valid_loss / len(val_loader.dataset)
        val_error.append(valid_loss)
        train_accuraacy.append( correct / total)
        valdation_accuracy.append(V_correct / V_total)
        if (V_correct / V_total) > max_val_acc:
            max_val_acc = V_correct / V_total
            torch.save(model.state_dict(), "./models/" + model_name + ".pth")

        print('\tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(train_loss, valid_loss))
        print('\tTrain Accuracy: %.3fd%% (%2d/%2d)\tValdation Accuracy: %.3fd%% (%2d/%2d) '% (100. * correct / total, correct, total, 100. * V_correct / V_total, V_correct, V_total))

    # torch.save(model, f'{model.__class__.__name__}.pt')
    print('Finished Training') 

In [9]:
def test(model, test_loader ,device, type=None):
    criterion = nn.CrossEntropyLoss()
    acc = 0.0
    test_loss = 0.0

    model.eval()
    with torch.no_grad():
        test_bar = tqdm(test_loader, file=sys.stdout)
        for test_data in test_bar:
            test_images, test_labels = test_data
            test_images, test_labels = test_images.to(device), test_labels.to(device)
            
            outputs, features = model(test_images)
            loss = criterion(outputs, test_labels)

            predict_y = torch.max(outputs, dim=1)[1]
            acc += torch.eq(predict_y, test_labels.to(device)).sum().item()
            test_loss += loss.item()
            test_bar.desc = "test"

    test_accurate = acc / test_num
    print('test_loss: %.3f  test_accuracy: %.3f' %(test_loss / test_steps, test_accurate * 100))
    return test_loss / test_steps, test_accurate * 100.

# Resnet34 3 channels

## Train Resnet34

In [11]:
model = ResNet(BasicBlock, [3, 4, 6, 3], num_classes=50)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
train_from_scratch(model, train_loader, val_loader, epochs=num_epochs, learning_rate=lr, device=device, model_name="best_resnet34_3channels")

valid epoch[1/10]: 100%|██████████| 4/4 [00:00<00:00,  4.87it/s]
	Training Loss: 3.883179 	Validation Loss: 3.881271
	Train Accuracy: 5.761d% (3648/63325)	Valdation Accuracy: 6.667d% (30/450) 
valid epoch[2/10]: 100%|██████████| 4/4 [00:00<00:00,  5.11it/s]
	Training Loss: 3.870358 	Validation Loss: 3.873805
	Train Accuracy: 7.207d% (4564/63325)	Valdation Accuracy: 6.667d% (30/450) 
valid epoch[3/10]: 100%|██████████| 4/4 [00:00<00:00,  4.98it/s]
	Training Loss: 3.853686 	Validation Loss: 3.852032
	Train Accuracy: 9.004d% (5702/63325)	Valdation Accuracy: 9.556d% (43/450) 
valid epoch[4/10]: 100%|██████████| 4/4 [00:00<00:00,  4.83it/s]
	Training Loss: 3.842217 	Validation Loss: 3.837766
	Train Accuracy: 10.141d% (6422/63325)	Valdation Accuracy: 10.444d% (47/450) 
valid epoch[5/10]: 100%|██████████| 4/4 [00:00<00:00,  4.95it/s]
	Training Loss: 3.834742 	Validation Loss: 3.860052
	Train Accuracy: 10.966d% (6944/63325)	Valdation Accuracy: 8.222d% (37/450) 
valid epoch[6/10]: 100%|████████

## Test Resnet34

In [12]:
test_model = ResNet(BasicBlock, [3, 4, 6, 3], num_classes=50).to(device)
test_model.load_state_dict(torch.load(os.getcwd() + "/" + "models/best_resnet34_3channels.pth"))
test_model.eval()
test(test_model, test_loader=test_loader, device=device)

test: 100%|██████████| 4/4 [00:00<00:00,  4.92it/s]
test_loss: 3.814  test_accuracy: 11.778


(3.8143553137779236, 11.777777777777777)

In [13]:
summary(model, input_size=(128, 3, 128, 128))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [128, 50]                 --
├─Conv2d: 1-1                            [128, 64, 128, 128]       1,728
├─BatchNorm2d: 1-2                       [128, 64, 128, 128]       128
├─ReLU: 1-3                              [128, 64, 128, 128]       --
├─MaxPool2d: 1-4                         [128, 64, 64, 64]         --
├─Sequential: 1-5                        [128, 64, 64, 64]         --
│    └─BasicBlock: 2-1                   [128, 64, 64, 64]         --
│    │    └─Conv2d: 3-1                  [128, 64, 64, 64]         36,864
│    │    └─BatchNorm2d: 3-2             [128, 64, 64, 64]         128
│    │    └─ReLU: 3-3                    [128, 64, 64, 64]         --
│    │    └─Conv2d: 3-4                  [128, 64, 64, 64]         36,864
│    │    └─BatchNorm2d: 3-5             [128, 64, 64, 64]         128
│    │    └─ReLU: 3-6                    [128, 64, 64, 64]         --
│

In [14]:
flops, params = get_model_complexity_info(model, (3, 128, 128), as_strings=True, print_per_layer_stat=True)

print(f"FLOPS: {flops}")
print(f"Parameters: {params}")

ResNet(
  21.3 M, 100.000% Params, 4.67 GMac, 99.877% MACs, 
  (conv1): Conv2d(1.73 k, 0.008% Params, 28.31 MMac, 0.605% MACs, 3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(128, 0.001% Params, 2.1 MMac, 0.045% MACs, 64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(0, 0.000% Params, 1.05 MMac, 0.022% MACs, inplace=True)
  (maxpool): MaxPool2d(0, 0.000% Params, 1.05 MMac, 0.022% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    221.95 k, 1.042% Params, 910.69 MMac, 19.459% MACs, 
    (0): BasicBlock(
      73.98 k, 0.347% Params, 303.56 MMac, 6.486% MACs, 
      (conv1): Conv2d(36.86 k, 0.173% Params, 150.99 MMac, 3.226% MACs, 64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, 0.001% Params, 524.29 KMac, 0.011% MACs, 64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(0, 0.000% 

# DynamicResnet34 3channels

## Train Dynamic Resnet34

In [15]:
dynamic_model = DynamicResNet(DynamicBasicBlock, [3, 4, 6, 3], num_classes=50)
device = "cuda" if torch.cuda.is_available() else "cpu"
dynamic_model.to(device)
train_from_scratch(dynamic_model, train_loader, val_loader, epochs=num_epochs, learning_rate=lr, device=device, model_name="best_dynamic_resnet34_3channels")

valid epoch[1/10]: 100%|██████████| 4/4 [00:00<00:00,  4.88it/s]
	Training Loss: 3.877905 	Validation Loss: 3.882598
	Train Accuracy: 6.433d% (4074/63325)	Valdation Accuracy: 5.778d% (26/450) 
valid epoch[2/10]: 100%|██████████| 4/4 [00:00<00:00,  4.90it/s]
	Training Loss: 3.854722 	Validation Loss: 3.843908
	Train Accuracy: 8.881d% (5624/63325)	Valdation Accuracy: 10.000d% (45/450) 
valid epoch[3/10]: 100%|██████████| 4/4 [00:00<00:00,  4.91it/s]
	Training Loss: 3.846102 	Validation Loss: 3.841099
	Train Accuracy: 9.709d% (6148/63325)	Valdation Accuracy: 10.889d% (49/450) 
valid epoch[4/10]: 100%|██████████| 4/4 [00:00<00:00,  4.87it/s]
	Training Loss: 3.836910 	Validation Loss: 3.832647
	Train Accuracy: 10.730d% (6795/63325)	Valdation Accuracy: 10.667d% (48/450) 
valid epoch[5/10]: 100%|██████████| 4/4 [00:00<00:00,  4.86it/s]
	Training Loss: 3.828022 	Validation Loss: 3.829474
	Train Accuracy: 11.597d% (7344/63325)	Valdation Accuracy: 11.778d% (53/450) 
valid epoch[6/10]: 100%|█████

## Test Dynamic Resnet34

In [16]:
test_model = DynamicResNet(DynamicBasicBlock, [3, 4, 6, 3], num_classes=50).to(device)
test_model.load_state_dict(torch.load(os.getcwd() + "/" + "models/best_dynamic_resnet34_3channels.pth"))
test_model.eval()
test(test_model, test_loader=test_loader, device=device)

test: 100%|██████████| 4/4 [00:00<00:00,  4.88it/s]
test_loss: 3.773  test_accuracy: 16.000


(3.7726109623908997, 16.0)

In [17]:
flops, params = get_model_complexity_info(dynamic_model, (3, 128, 128), as_strings=True, print_per_layer_stat=True)

print(f"FLOPS: {flops}")
print(f"Parameters: {params}")

DynamicResNet(
  21.24 M, 98.597% Params, 4.35 GMac, 99.848% MACs, 
  (conv1): DynamicConv(
    21, 0.000% Params, 49.18 KMac, 0.001% MACs, 
    (attention): attention2d(
      21, 0.000% Params, 49.18 KMac, 0.001% MACs, 
      (avgpool): AdaptiveAvgPool2d(0, 0.000% Params, 49.15 KMac, 0.001% MACs, output_size=1)
      (cal): Sequential(
        21, 0.000% Params, 24.0 Mac, 0.000% MACs, 
        (0): Conv2d(9, 0.000% Params, 9.0 Mac, 0.000% MACs, 3, 3, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): ReLU(0, 0.000% Params, 3.0 Mac, 0.000% MACs, )
        (2): Conv2d(12, 0.000% Params, 12.0 Mac, 0.000% MACs, 3, 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
    )
  )
  (bn1): BatchNorm2d(128, 0.001% Params, 2.1 MMac, 0.048% MACs, 64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(0, 0.000% Params, 1.05 MMac, 0.024% MACs, inplace=True)
  (maxpool): MaxPool2d(0, 0.000% Params, 1.05 MMac, 0.024% MACs, kernel_size=3, stride=2, padding=1,

In [18]:
summary(dynamic_model, input_size=(128, 3, 128, 128))

Layer (type:depth-idx)                             Output Shape              Param #
DynamicResNet                                      [128, 50]                 --
├─DynamicConv: 1-1                                 [128, 64, 128, 128]       6,912
│    └─attention2d: 2-1                            [128, 4]                  --
│    │    └─AdaptiveAvgPool2d: 3-1                 [128, 3, 1, 1]            --
│    │    └─Sequential: 3-2                        [128, 4, 1, 1]            21
├─BatchNorm2d: 1-2                                 [128, 64, 128, 128]       128
├─ReLU: 1-3                                        [128, 64, 128, 128]       --
├─MaxPool2d: 1-4                                   [128, 64, 64, 64]         --
├─Sequential: 1-5                                  [128, 64, 64, 64]         --
│    └─DynamicBasicBlock: 2-2                      [128, 64, 64, 64]         --
│    │    └─DynamicConv: 3-3                       [128, 64, 64, 64]         152,064
│    │    └─BatchNorm2d: 3

# Resnet34 1 channels

## Train

In [19]:
train_data = CustomImageDataset(txt_file="train.txt", img_dir=img_dir, transform=transform_train1, convert="L")
val_data = CustomImageDataset(txt_file="val.txt", img_dir=img_dir, transform=transform_train1, convert="L")
test_data = CustomImageDataset(txt_file="test.txt", img_dir=img_dir, transform=transform_test1, convert="L")

# DataLoader
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)

print(f"Training Set length:{len(train_data)}, Validating Set length:{len(val_data)}")

test_num = len(test_data)
test_steps = len(test_loader)

Training Set length:63325, Validating Set length:450


In [20]:
model1 = ResNet(BasicBlock, [3, 4, 6, 3], num_classes=50, channels=1)
device = "cuda" if torch.cuda.is_available() else "cpu"
model1.to(device)
train_from_scratch(model1, train_loader, val_loader, epochs=num_epochs, learning_rate=lr, device=device, model_name="best_resnet34_1channels")

valid epoch[1/10]: 100%|██████████| 4/4 [00:00<00:00,  6.19it/s]
	Training Loss: 3.903598 	Validation Loss: 3.901765
	Train Accuracy: 3.523d% (2231/63325)	Valdation Accuracy: 3.333d% (15/450) 
valid epoch[2/10]: 100%|██████████| 4/4 [00:00<00:00,  6.10it/s]
	Training Loss: 3.892020 	Validation Loss: 3.916335
	Train Accuracy: 4.962d% (3142/63325)	Valdation Accuracy: 2.444d% (11/450) 
valid epoch[3/10]: 100%|██████████| 4/4 [00:00<00:00,  6.10it/s]
	Training Loss: 3.881632 	Validation Loss: 3.885682
	Train Accuracy: 6.077d% (3848/63325)	Valdation Accuracy: 6.222d% (28/450) 
valid epoch[4/10]: 100%|██████████| 4/4 [00:00<00:00,  6.14it/s]
	Training Loss: 3.873114 	Validation Loss: 3.880978
	Train Accuracy: 6.972d% (4415/63325)	Valdation Accuracy: 6.667d% (30/450) 
valid epoch[5/10]: 100%|██████████| 4/4 [00:00<00:00,  6.13it/s]
	Training Loss: 3.865336 	Validation Loss: 3.847461
	Train Accuracy: 7.796d% (4937/63325)	Valdation Accuracy: 10.222d% (46/450) 
valid epoch[6/10]: 100%|██████████

## Test

In [21]:
test_model = ResNet(BasicBlock, [3, 4, 6, 3], num_classes=50, channels=1).to(device)
test_model.load_state_dict(torch.load(os.getcwd() + "/" + "models/best_resnet34_1channels.pth"))
test_model.eval()
test(test_model, test_loader=test_loader, device=device)

test: 100%|██████████| 4/4 [00:00<00:00,  6.18it/s]
test_loss: 3.847  test_accuracy: 9.778


(3.8471043705940247, 9.777777777777779)

In [22]:
summary(model1, input_size=(128, 1, 128, 128))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [128, 50]                 --
├─Conv2d: 1-1                            [128, 64, 128, 128]       576
├─BatchNorm2d: 1-2                       [128, 64, 128, 128]       128
├─ReLU: 1-3                              [128, 64, 128, 128]       --
├─MaxPool2d: 1-4                         [128, 64, 64, 64]         --
├─Sequential: 1-5                        [128, 64, 64, 64]         --
│    └─BasicBlock: 2-1                   [128, 64, 64, 64]         --
│    │    └─Conv2d: 3-1                  [128, 64, 64, 64]         36,864
│    │    └─BatchNorm2d: 3-2             [128, 64, 64, 64]         128
│    │    └─ReLU: 3-3                    [128, 64, 64, 64]         --
│    │    └─Conv2d: 3-4                  [128, 64, 64, 64]         36,864
│    │    └─BatchNorm2d: 3-5             [128, 64, 64, 64]         128
│    │    └─ReLU: 3-6                    [128, 64, 64, 64]         --
│  

In [23]:
flops, params = get_model_complexity_info(model1, (1, 128, 128), as_strings=True, print_per_layer_stat=True)

print(f"FLOPS: {flops}")
print(f"Parameters: {params}")

ResNet(
  21.3 M, 100.000% Params, 4.66 GMac, 99.877% MACs, 
  (conv1): Conv2d(576, 0.003% Params, 9.44 MMac, 0.202% MACs, 1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(128, 0.001% Params, 2.1 MMac, 0.045% MACs, 64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(0, 0.000% Params, 1.05 MMac, 0.022% MACs, inplace=True)
  (maxpool): MaxPool2d(0, 0.000% Params, 1.05 MMac, 0.022% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    221.95 k, 1.042% Params, 910.69 MMac, 19.538% MACs, 
    (0): BasicBlock(
      73.98 k, 0.347% Params, 303.56 MMac, 6.513% MACs, 
      (conv1): Conv2d(36.86 k, 0.173% Params, 150.99 MMac, 3.239% MACs, 64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, 0.001% Params, 524.29 KMac, 0.011% MACs, 64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(0, 0.000% Para

# DynamicResnet 1 channels

## Train

In [24]:
dynamic_model1 = DynamicResNet(DynamicBasicBlock, [3, 4, 6, 3], num_classes=50, channels=1)
device = "cuda" if torch.cuda.is_available() else "cpu"
dynamic_model1.to(device)
train_from_scratch(dynamic_model1, train_loader, val_loader, epochs=num_epochs, learning_rate=lr, device=device, model_name="best_dynamic_resnet34_1channels")

valid epoch[1/10]: 100%|██████████| 4/4 [00:00<00:00,  5.80it/s]
	Training Loss: 3.893568 	Validation Loss: 3.902633
	Train Accuracy: 4.698d% (2975/63325)	Valdation Accuracy: 3.778d% (17/450) 
valid epoch[2/10]: 100%|██████████| 4/4 [00:00<00:00,  5.87it/s]
	Training Loss: 3.876602 	Validation Loss: 3.880385
	Train Accuracy: 6.685d% (4233/63325)	Valdation Accuracy: 6.000d% (27/450) 
valid epoch[3/10]: 100%|██████████| 4/4 [00:00<00:00,  5.81it/s]
	Training Loss: 3.868246 	Validation Loss: 3.870107
	Train Accuracy: 7.468d% (4729/63325)	Valdation Accuracy: 7.111d% (32/450) 
valid epoch[4/10]: 100%|██████████| 4/4 [00:00<00:00,  5.85it/s]
	Training Loss: 3.862120 	Validation Loss: 3.859920
	Train Accuracy: 8.158d% (5166/63325)	Valdation Accuracy: 8.444d% (38/450) 
valid epoch[5/10]: 100%|██████████| 4/4 [00:00<00:00,  5.84it/s]
	Training Loss: 3.854429 	Validation Loss: 3.851288
	Train Accuracy: 8.916d% (5646/63325)	Valdation Accuracy: 9.333d% (42/450) 
valid epoch[6/10]: 100%|██████████|

## Test

In [25]:
test_model = DynamicResNet(DynamicBasicBlock, [3, 4, 6, 3], num_classes=50, channels=1).to(device)
test_model.load_state_dict(torch.load(os.getcwd() + "/" + "models/best_dynamic_resnet34_1channels.pth"))
test_model.eval()
test(test_model, test_loader=test_loader, device=device)

test: 100%|██████████| 4/4 [00:00<00:00,  5.83it/s]
test_loss: 3.805  test_accuracy: 13.111


(3.804931402206421, 13.111111111111112)

In [26]:
flops, params = get_model_complexity_info(dynamic_model1, (1, 128, 128), as_strings=True, print_per_layer_stat=True)

print(f"FLOPS: {flops}")
print(f"Parameters: {params}")

DynamicResNet(
  21.24 M, 98.618% Params, 4.35 GMac, 99.849% MACs, 
  (conv1): DynamicConv(
    5, 0.000% Params, 16.39 KMac, 0.000% MACs, 
    (attention): attention2d(
      5, 0.000% Params, 16.39 KMac, 0.000% MACs, 
      (avgpool): AdaptiveAvgPool2d(0, 0.000% Params, 16.38 KMac, 0.000% MACs, output_size=1)
      (cal): Sequential(
        5, 0.000% Params, 6.0 Mac, 0.000% MACs, 
        (0): Conv2d(1, 0.000% Params, 1.0 Mac, 0.000% MACs, 1, 1, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): ReLU(0, 0.000% Params, 1.0 Mac, 0.000% MACs, )
        (2): Conv2d(4, 0.000% Params, 4.0 Mac, 0.000% MACs, 1, 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
    )
  )
  (bn1): BatchNorm2d(128, 0.001% Params, 2.1 MMac, 0.048% MACs, 64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(0, 0.000% Params, 1.05 MMac, 0.024% MACs, inplace=True)
  (maxpool): MaxPool2d(0, 0.000% Params, 1.05 MMac, 0.024% MACs, kernel_size=3, stride=2, padding=1, dilat

In [27]:
summary(dynamic_model1, input_size=(128, 1, 128, 128))

Layer (type:depth-idx)                             Output Shape              Param #
DynamicResNet                                      [128, 50]                 --
├─DynamicConv: 1-1                                 [128, 64, 128, 128]       2,304
│    └─attention2d: 2-1                            [128, 4]                  --
│    │    └─AdaptiveAvgPool2d: 3-1                 [128, 1, 1, 1]            --
│    │    └─Sequential: 3-2                        [128, 4, 1, 1]            5
├─BatchNorm2d: 1-2                                 [128, 64, 128, 128]       128
├─ReLU: 1-3                                        [128, 64, 128, 128]       --
├─MaxPool2d: 1-4                                   [128, 64, 64, 64]         --
├─Sequential: 1-5                                  [128, 64, 64, 64]         --
│    └─DynamicBasicBlock: 2-2                      [128, 64, 64, 64]         --
│    │    └─DynamicConv: 3-3                       [128, 64, 64, 64]         152,064
│    │    └─BatchNorm2d: 3-